# Download data for events
This part of the code will be used to download event data.

It is based on Josh Russell's **fetch_EVENTS**.

*william b hawley april 2020*

In [1]:
from config import *
import os

import numpy as np
import math
import matplotlib.pyplot as plt

import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from obspy.io.sac import SACTrace

from datetime import datetime
import calendar

import pyproj

%matplotlib inline
%load_ext autoreload
%autoreload

In [2]:
# Check to see if data directory exists
if not os.path.exists(DataDir):
    os.makedirs(DataDir)

# load the client
client = Client(webservice)

In [3]:
# Load event catalogue
t1 = UTCDateTime(tstart)
t2 = UTCDateTime(tend)
catIRIS = client.get_events(starttime=t1, endtime=t2, minmagnitude=minMag)

print((str(len(catIRIS))+" events in catalogue."))

# Load stations
inventory = client.get_stations(network=network, station=','.join(StaList), channel=','.join(ChanList), starttime=t1, endtime=t2)

2120 events in catalogue.


In [4]:
# New section to get even azimuthal coverage

# we will store event info here
EvtList = []
EvtListUse = []

# and number of earthquakes per bin greater than some value here
NumLargeEqs = [0] * NAziBin

for iev in range(0,len(catIRIS)):
    if isCMT_params: 
        if isCentroid:
            ior = 1
        else:
            ior = 0
        
        # if searching CMT catalogue, pull out necessary time info
        time = catIRIS[iev].origins[0].time
        date = datetime.strptime(str(time),'%Y-%m-%dT%H:%M:%S.%fZ')
        month = calendar.month_abbr[date.month].lower()
        year = str(date.year)
        
        # in case time is not exactly the same...
        time1 = str(time-50) #+/- 50 seconds
        time2 = str(time+50)
        catCMT = obspy.read_events("https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/"+year+"/"+month+""+year[2:4]+".ndk")
        cat_filt = catCMT.filter('time > '+time1, 'time < '+time2, 'magnitude >= '+str(minMag-1))
        if len(cat_filt)==0:
            #print('Cannot find event in GCMT catalogue... using IRIS')
            cat = catIRIS[iev].copy()
            ior = 0
        else:
            cat = cat_filt[0].copy()
    else:
        cat = catIRIS[iev].copy()
        ior = 0
    
    # earthquake parameters
    evT1 = cat.origins[ior].time
    evT2 = evT1 + trLen
    evdp = cat.origins[ior].depth
    evla = cat.origins[ior].latitude
    evlo = cat.origins[ior].longitude
    mag = cat.magnitudes[0].mag
    
    # date for naming folders
    date = datetime.strptime(str(evT1),'%Y-%m-%dT%H:%M:%S.%fZ')
    evName = date.strftime('%Y%m%d%H%M')
    
    # calculate backazimuth
    geodesic = pyproj.Geod(ellps='WGS84')
    azi,bazi,dist = geodesic.inv(evlo, evla, ArrayLoc[1], ArrayLoc[0])

    # put in back azimuth bin
    baziNonNeg = bazi
    # to make the bins clockwise from north
    if baziNonNeg < 0:
        baziNonNeg = 360 + bazi
    bazBin = (baziNonNeg/360)*NAziBin
    bazBin = math.floor(bazBin)

    # save all in this structure
    EvInfo = [evName,evT1,bazi,bazBin,mag]
    EvtList.append(EvInfo)
    # and if eq is larger than 6.5, add one to the correct bin
    if mag >= LargeEqCutoff:
        NumLargeEqs[bazBin] += 1
        EvtListUse.append([evName,evT1])

In [5]:
#print(EvInfo)
#print(NumLargeEqs)
#print(LargeEqCutoff)

# need to functionalize this
def FindMoreEvents(ibin,Neq,EvtList,EvtListUseFunc,LargeEqCutoff):
    #EventListUseBin = []
    Nfound = 0
    # start with largest mag, and work our way down
    for mdiff in np.arange(0.1,9,0.1):
        msearch = LargeEqCutoff - mdiff
        #while Nfound < Neq:
        # loop through eqs
        for ev in EvtList:
            # use events in the right bin
            if ev[3] != ibin:
                #print('not right bin, '+str(ev[3])+' is not '+str(ibin))
                continue 
            # use events of right magnitude
            if round(ev[4],1) != msearch:
                continue
            # now check to see if within some num of hours (2?)of another event already 
            trange = 2*60*60 # (utcdatetime uses 1 as a second... add 2 hours here)
            for evTest in EvtListUse:
                testTime = evTest[1]
                if ev[1] < testTime+trange and ev[1] > testTime-trange:
                    continue
            # if it has passed all these tests, we should add it to the event list
            print('Using event '+ev[0]+', M'+str(ev[4])+' for bin '+str(ibin))
            EvtListUseFunc.append([ev[0],ev[1]])
            Nfound += 1
            #try here -- not while loop
            if Nfound == Neq:
                print('done with bin '+str(ibin))
                return EvtListUseFunc
    

# find number of eqs in bin with largest number of eq larger than the cutoff above
maxeq = max(NumLargeEqs)
maxbin = NumLargeEqs.index(maxeq)

# now loop through all the other azi bins
for ibin in range(0,NAziBin):
    # get how many eqs we need for this bin
    Neq = maxeq - NumLargeEqs[ibin]
    if Neq == 0:
        continue
    # and keep track of how many we have found

    
    EvtListUseNew = FindMoreEvents(ibin,Neq,EvtList,EvtListUse,LargeEqCutoff)
    
            


# want other bins to have similar number of these
maxindex = NumLargeEqs.index(maxeq)

Using event 201306162139, M6.12 for bin 0
Using event 201301081416, M5.81 for bin 0
Using event 201212231331, M5.77 for bin 0
Using event 201307031921, M5.69 for bin 0
Using event 201211070626, M5.56 for bin 0
Using event 201210210125, M5.58 for bin 0
Using event 201210141013, M5.56 for bin 0
Using event 201308041322, M5.46 for bin 0
Using event 201307081530, M5.54 for bin 0
Using event 201301202332, M5.47 for bin 0
Using event 201310011941, M5.32 for bin 0
Using event 201309170409, M5.33 for bin 0
Using event 201309161501, M5.3 for bin 0
Using event 201306211034, M5.32 for bin 0
Using event 201304061126, M5.35 for bin 0
Using event 201304020059, M5.27 for bin 0
Using event 201301202126, M5.3 for bin 0
Using event 201210252305, M5.3 for bin 0
Using event 201210210010, M5.35 for bin 0
done with bin 0
Using event 201309050401, M5.98 for bin 1
Using event 201304300625, M5.81 for bin 1
Using event 201304021434, M5.85 for bin 1
Using event 201211031258, M5.67 for bin 1
Using event 201302180

In [6]:
#EventListOld = EvtListUse
print(len(EvtListUse))
#print(len(EventListOld))
#EventListOld = []
#a = EventListOld
#print(len(a))
#EvtListUseNew

160


In [7]:
# Get event times
f = open(EventsFileName,'w')
for iev in range(0,len(catIRIS)):
    if isCMT_params: 
        if isCentroid:
            ior = 1
        else:
            ior = 0
        
        # if searching CMT catalogue, pull out necessary time info
        time = catIRIS[iev].origins[0].time
        date = datetime.strptime(str(time),'%Y-%m-%dT%H:%M:%S.%fZ')
        month = calendar.month_abbr[date.month].lower()
        year = str(date.year)
        
        # in case time is not exactly the same...
        time1 = str(time-50) #+/- 50 seconds
        time2 = str(time+50)
        catCMT = obspy.read_events("https://www.ldeo.columbia.edu/~gcmt/projects/CMT/catalog/NEW_MONTHLY/"+year+"/"+month+""+year[2:4]+".ndk")
        cat_filt = catCMT.filter('time > '+time1, 'time < '+time2, 'magnitude >= '+str(minMag-1))
        if len(cat_filt)==0:
            print('Cannot find event in GCMT catalogue... using IRIS')
            cat = catIRIS[iev].copy()
            ior = 0
        else:
            cat = cat_filt[0].copy()
    else:
        cat = catIRIS[iev].copy()
        ior = 0
    
    # earthquake parameters
    evT1 = cat.origins[ior].time
    evT2 = evT1 + trLen
    evdp = cat.origins[ior].depth
    evla = cat.origins[ior].latitude
    evlo = cat.origins[ior].longitude
    mag = cat.magnitudes[0].mag
    
    # date for naming folders
    date = datetime.strptime(str(evT1),'%Y-%m-%dT%H:%M:%S.%fZ')
    evName = date.strftime('%Y%m%d%H%M')
    evDir = EventsDataDir + evName + '/'
    f.write(evName+'\n')
    if not os.path.exists(evDir):
        os.makedirs(evDir)
    print('Working on event '+evName)
    #else: #...leave this out. Can download new station data for same event. 
        #print('Event '+evName+' exists, skipping...')
        #continue
        
    # loop through stations
    #for ista in range(0,len(inventoryX[0])):
    for ista in range(0,len(inventory[0])):
        stel = inventory[0].stations[ista].elevation
        stla = inventory[0].stations[ista].latitude
        stlo = inventory[0].stations[ista].longitude
        station = inventory[0].stations[ista].code
        vals = gps2dist_azimuth(lat1=stla, lon1=stlo, lat2=evla, lon2=evlo)
        dist = vals[0]
        baz = vals[1]
        az = vals[2]
        gcarc = locations2degrees(lat1=stla, long1=stlo, lat2=evla, long2=evlo)
        
        # Loop through components... 
        for comp in ChanList:

            
            # need to look for backups for some stations, particularly for 7D
            if comp == 'BDH':
                backupComp = 'BXH'
                backupComp2 = 'BXH'
                backupComp3 = 'BXH'
            elif comp == 'LHZ':
                backupComp = 'LXZ'
                backupComp2 = 'BXZ'
                backupComp3 = 'BHZ'
            elif comp == 'LH1':
                backupComp = 'LX1'
                backupComp2 = 'BX1'
                backupComp3 = 'BH1'
            elif comp == 'LH2':
                backupComp = 'LX2'
                backupComp2 = 'BXH'
                backupComp3 = 'BH2'
                
            sac_out = evDir + evName + '.' + network + '.' + station + '.' + comp + '.sac'
            if os.path.exists(sac_out):
                print("File "+sac_out+" exists; skipping")
                continue            
            try:
                st = client.get_waveforms(network=network, station=station, location='*', channel=comp, starttime=evT1, endtime=evT2, attach_response=True)
            except Exception:
                try:
                    st = client.get_waveforms(network=network, station=station, location='*', channel=backupComp, starttime=evT1, endtime=evT2, attach_response=True)
                    print('Using comp '+backupComp+' for station '+station)
                except Exception:
                    try:
                        st = client.get_waveforms(network=network, station=station, location='*', channel=backupComp2, starttime=evT1, endtime=evT2, attach_response=True)
                        print('Using comp '+backupComp2+' for station '+station)
                    except Exception:
                        try:
                            st = client.get_waveforms(network=network, station=station, location='*', channel=backupComp3, starttime=evT1, endtime=evT2, attach_response=True)
                            print('Using comp '+backupComp3+' for station '+station)
                        except Exception:
                            print('Missing data for station: ',station,'Comps:',comp,backupComp,backupComp2,backupComp3)
                        continue
                
            # if BXH, change back ... doesn't matter; we downsample to 1 Hz anyway
            if comp == 'BXH':
                comp = 'BDH'
                
            # check for gaps
            if len(st) > 1:
                st.merge(method=1, fill_value=0)
            sr = st[0].stats.sampling_rate
            st.remove_response(output="DISP", zero_mean=True, taper=True, taper_fraction=0.05, pre_filt=[LoFreq1,LoFreq2,sr/3,sr/2], water_level=60)
            st.trim(starttime=evT1, endtime=evT2, pad=True, nearest_sample=False, fill_value=0)
            st.detrend(type='demean')
            st.detrend(type='linear')
            st.taper(type='cosine',max_percentage=0.05)
            
            # downsample the trace
            if isDownsamp==1:
                st.filter('lowpass', freq=0.4*srNew, zerophase=True) #anti-alias
                st.resample(sampling_rate=srNew)
                st.detrend(type='demean')
                st.detrend(type='linear')
                st.taper(type='cosine',max_percentage=0.05)
                
            # convert to SAC
            sac = SACTrace.from_obspy_trace(st[0])
            sac.stel = stel
            sac.stla = stla
            sac.stlo = stlo
            sac.evdp = evdp
            sac.evla = evla
            sac.evlo = evlo
            sac.mag = mag
            sac.dist = dist
            sac.az = az
            sac.baz = baz
            sac.gcarc = gcarc
            sac.kcmpnm = comp
            

            sac.write(sac_out)
            
f.close()

Working on event 201310052050
Missing data for station:  G02B Comps: LHZ LXZ BXZ BHZ
Missing data for station:  G02B Comps: LH1 LX1 BX1 BH1
Missing data for station:  G02B Comps: LH2 LX2 BXH BH2
Missing data for station:  G02B Comps: BDH BXH BXH BXH
Missing data for station:  G03B Comps: LHZ LXZ BXZ BHZ
Missing data for station:  G03B Comps: LH1 LX1 BX1 BH1
Missing data for station:  G03B Comps: LH2 LX2 BXH BH2
Missing data for station:  G03B Comps: BDH BXH BXH BXH
Missing data for station:  G04B Comps: LHZ LXZ BXZ BHZ
Missing data for station:  G04B Comps: LH1 LX1 BX1 BH1
Missing data for station:  G04B Comps: LH2 LX2 BXH BH2
Missing data for station:  G04B Comps: BDH BXH BXH BXH
Missing data for station:  G05B Comps: LHZ LXZ BXZ BHZ
Missing data for station:  G05B Comps: LH1 LX1 BX1 BH1
Missing data for station:  G05B Comps: LH2 LX2 BXH BH2
Missing data for station:  G05B Comps: BDH BXH BXH BXH
Missing data for station:  G10B Comps: LHZ LXZ BXZ BHZ
Missing data for station:  G10B Com

KeyboardInterrupt: 